# Text Recognition

In [1]:
#!pip install google-cloud-vision

In [8]:
#import libraries
import pandas as pd
import numpy as np
import json
import requests
import re
import pyautogui
from thefuzz import fuzz

## Attempt 1: Google OCR -- detect text in images

In [9]:
# def detect_text(path):
#     """Detects text in the file."""
#     from google.oauth2 import service_account
#     from google.cloud import vision

#     credentials = service_account.Credentials.from_service_account_file('divine-outlet-408519-ead34ab34a37.json')

#     client = vision.ImageAnnotatorClient(credentials=credentials)

#     with open(path, "rb") as image_file:
#         content = image_file.read()

#     image = vision.Image(content=content)

#     response = client.text_detection(image=image)
#     texts = response.text_annotations

#     return([text.description for text in texts])


#     if response.error.message:
#         raise Exception(
#             "{}\nFor more info on error messages, check: "
#             "https://cloud.google.com/apis/design/errors".format(response.error.message)
#         )


In [10]:
# def strRecognition(list):
#     # Extracting product names and quantities
#   description_section = re.search(r'Description\n(.*?)\n(?:P\.O\. No\.|Date)', list[0], re.DOTALL).group(1)
#   product_info = re.findall(r'(\d{1,4} [\w\s\*,\.]+) (\d+)', description_section)

#   products = []
#   for quantity, name, _ in product_info:
#       products.append({
#           'Product': name.strip(),
#           'Quantity': quantity.replace(',', '')
#       })

#   # Extracting rates
#   rates_section = re.search(r'Date\n[\d/]+\n([\d\s.$]+)\nInvoice', str(list[0])).group(1)
#   rates = re.findall(r'[\d.]+', rates_section)

#   # Assigning rates to products
#   for i, rate in enumerate(rates):
#       products[i]['Rate'] = rate

#   # Printing extracted information
#   for product in products:
#       print(f"Product: {product['Product']}, Quantity: {product['Quantity']}, Rate: {product['Rate']}")

#   # Creating a DataFrame
#   df = pd.DataFrame(products)

#   # Printing DataFrame
#   return df

## Attempt 2: Google OCR -- detect text in files

This function is able to detect texts by blocks, which may be more suitable for this case with tables as inputs.

In [11]:
# def async_detect_document(gcs_source_uri, gcs_destination_uri):
#     """OCR with PDF/TIFF as source files on GCS"""
#     import json
#     import re
#     from google.cloud import vision
#     from google.cloud import storage

#     # Supported mime_types are: 'application/pdf' and 'image/tiff'
#     mime_type = "image/tiff"

#     # How many pages should be grouped into each json output file.
#     batch_size = 1

#     credentials = service_account.Credentials.from_service_account_file('credential/divine-outlet-408519-ead34ab34a37.json')
#     client = vision.ImageAnnotatorClient(credentials=credentials)

#     feature = vision.Feature(type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

#     gcs_source = vision.GcsSource(uri=gcs_source_uri)
#     input_config = vision.InputConfig(gcs_source=gcs_source, mime_type=mime_type)

#     gcs_destination = vision.GcsDestination(uri=gcs_destination_uri)
#     output_config = vision.OutputConfig(
#         gcs_destination=gcs_destination, batch_size=batch_size
#     )

#     async_request = vision.AsyncAnnotateFileRequest(
#         features=[feature], input_config=input_config, output_config=output_config
#     )

#     operation = client.async_batch_annotate_files(requests=[async_request])

#     print("Waiting for the operation to finish.")
#     operation.result(timeout=420)

#     # Once the request has completed and the output has been
#     # written to GCS, we can list all the output files.
#     storage_client = storage.Client()

#     match = re.match(r"gs://([^/]+)/(.+)", gcs_destination_uri)
#     bucket_name = match.group(1)
#     prefix = match.group(2)

#     bucket = storage_client.get_bucket(bucket_name)

#     # List objects with the given prefix, filtering out folders.
#     blob_list = [
#         blob
#         for blob in list(bucket.list_blobs(prefix=prefix))
#         if not blob.name.endswith("/")
#     ]
#     print("Output files:")
#     for blob in blob_list:
#         print(blob.name)

#     # Process the first output file from GCS.
#     # Since we specified batch_size=2, the first response contains
#     # the first two pages of the input file.
#     output = blob_list[0]

#     json_string = output.download_as_bytes().decode("utf-8")
#     response = json.loads(json_string)

#     # The actual response for the first page of the input file.
#     first_page_response = response["responses"][0]
#     annotation = first_page_response["fullTextAnnotation"]

#     # Here we print the full text from the first page.
#     # The response contains more information:
#     # annotation/pages/blocks/paragraphs/words/symbols
#     # including confidence scores and bounding boxes
#     print("Full text:\n")
#     print(annotation["text"])


## Attempt 3: OCR Space

In [12]:
def ocrSpaceFile(filename, api_key, overlay=False, language='eng', OCREngine = 2):

    payload = {'isOverlayRequired': overlay,
               'apikey': api_key,
               'language': language,
               'isTable': True,
               'OCREngine': OCREngine,
               #'detectOrientation':True
               }
    with open(filename, 'rb') as f:
        r = requests.post('https://api.ocr.space/parse/image',
                          files={filename: f},
                          data=payload,
                          )
    return r.content.decode()

# Extract information from the converted text

In [13]:
#get the result from API of best performance
def parsedText(filename, language = 'eng', OCREngine =2 ):
    response = ocrSpaceFile(filename, language = language,  OCREngine =OCREngine)
    data = json.loads(response)
    #there are two type of converting result of the OCR API
    #one is putting the result in one string
    text = data['ParsedResults'][0]['ParsedText']
    #the other is writing them in separate lines
    separateLines = []
    for i in range(len(data['ParsedResults'][0]['TextOverlay']['Lines'])):

        separateLines.append(data['ParsedResults'][0]['TextOverlay']['Lines'][i]['LineText'])
    
    return text, separateLines

In [14]:
def extractString(text):
    date=re.findall(r'\d{1,2}/\d{1,2}/\d{4}',text)[0]
    invoiceNo = re.findall('(?<![A-Z]{2}\s)\d{5,}', text)[0]
    companyName = re.findall('(?!R&JY).+INC', text)[0]
    
    return date, invoiceNo, companyName

Vendor Number is the initials of the company name, which is required when entering the invoice info

In [15]:
def extractInitials(text):
    # Split the text into words
    words = text.split()
    
    # Extract the first letter of each word and convert it to uppercase
    initials = [word[0].upper() for word in words if word]
    
    # Join the first three initials
    return ''.join(initials[:3])

In [16]:
#integrate all these functions into one
def readStrings(text):
    date, invoiceNo, companyName = extractString(text)
    companyInitial = extractInitials(companyName)
    return date, invoiceNo, companyInitial

# Extract item informations

In [18]:
def extractLines(text, marker_pattern):
    # Search for the marker using the regex pattern
    marker_match = re.search(marker_pattern, text, re.IGNORECASE)
    
    # If the marker isn't found, return an empty list
    if not marker_match:
        return []

    # Find the part of the text after the marker
    text_after_marker = text[marker_match.end():]

    # Split the text into lines
    lines = text_after_marker.strip().split('\n')

    # Define a function to check if a line starts with a number
    def starts_with_number(s):
        return s.lstrip()[0].isdigit()

    # Filter and return the lines that start with a number and contain 2 to 4 tabs
    return [line for line in lines if starts_with_number(line) and 2 <= line.count('\t')]






In [19]:
# Extract and print the lines
#if any words in the markers pattern is detected, identify that row as the header of the table
def itemInfo(text, marker_pattern = r'(数量|quantity|摘要|description|重量|weight|rate)'):
    
    extracted_lines = extractLines(text, r'(数量|quantity|摘要|description|重量|weight|rate)')
    amounts=[]
    rates = []
    items = []
    for line in extracted_lines:
                #amount is the first number in a row
                #the thousands separator is "," ,but it may be recognized as a decimal point
                amount_match = re.findall('\d{0,3}[.,]?\d+', line)[0] 
                #if nothing is fetched, return the line and let the user manually enter the amount
                amount = amount_match[0] if amount_match else input(f"Enter amount for line '{line}': ")
                #rate is the second number on the invoice, it must contain a decimal separator
                rate_match =re.findall(amount+'.*?(\t\d{0,3}[.,]{0,1}\d+[\.,]\d+)', line)[0]
                #if nothing is fetched, return the line and let the user manually enter the rate
                rate = rate_match[0] if rate_match else input(f"Enter rate for line '{line}': ")
                rate = re.sub('[.,]','.', rate)
                rate = rate.replace('\t', '')
                #item is the rest part in a row
                item_match = re.findall(amount+'?(.*)'+rate, line)[0]
                #if nothing is fetched, return the line and let the user manually enter the item
                item = item_match[0].strip() if item_match else input(f"Enter item for line '{line}': ")
                item = re.findall(r'\t{0,1}(.+?)\t$', item)[0]
                item = item.replace('\t', ' ')
                amount = re.sub('[.,]',',', amount)
                amounts.append(amount)
                rates.append(rate)
                items.append(item)

    df = pd.DataFrame({'amount':amounts,
                          'item':items,
                          'rate': rates})
    return df

# Add item code to the item information dataframe

In [20]:
#import the matching rule
#rule = pd.read_excel('data\matchingRules\InventorySummaryReport.xls')

In [21]:
def matchingCode(item,rule):
    score=np.repeat(0,len(rule))
    for i in range(len(rule)):
        score[i]=fuzz.ratio(item, rule['項目名稱_x000D_\nItem Name'][i])
        idxMax=np.argmax(np.array(score))
    return rule['項目編號_x000D_\nItem No.'][idxMax], rule['項目名稱_x000D_\nItem Name'][idxMax]

# Put it all together


In [22]:
#define a function combines all the functions together
def readFile(filename,language = 'cht', OCREngine =2):
    text, separateLines = parsedText(filename, language = language, OCREngine =OCREngine)
    date,invoiceNo, companyName=readStrings(text)
    df=itemInfo(text)
    df['code'] = df['item'].apply(lambda x: matchingCode(x, rule)[0])
    df['matched_rule'] = df['item'].apply(lambda x: matchingCode(x, rule)[1])
    
    return date,invoiceNo, companyName, df